# Demo: Run Multiple Intent Experiments

In [1]:
!rm -rf ./test

In [2]:
%%capture
from leaderboard.utils import experiment
from leaderboard.nlu import exp_0_stratify_intents
import copy

## Create Configurations Programmatically

In [3]:
base_config = exp_0_stratify_intents.Config(
    train_exclusion_fraction=0.,
    remove_data_for_intents_with_num_examples_below=2,
    data=experiment.DataConfig(
        name="examples-rules-nlu",
        data_path="../../examples/rules/data/nlu.yml",
    ),
    model=experiment.ModelConfig(),
    clear_rasa_cache=True,
)

In [4]:
model_variations = {
    "interesting-config-name1" : "./config2.yml" ,
    "another-very-interesting-config-because-reasons" : "./config1.yml" ,
}
model_variations = [ {("model","name") : key, ("model", "config_path") : value} for key, value in model_variations.items() ]

In [5]:
model_variations

[{('model', 'name'): 'interesting-config-name1',
  ('model', 'config_path'): './config2.yml'},
 {('model', 'name'): 'another-very-interesting-config-because-reasons',
  ('model', 'config_path'): './config1.yml'}]

In [6]:
seed_variations = [ {('exclusion_seed',) : seed} for seed in [1,2,3,4] ]

In [7]:
configs = experiment.create_variations([base_config], model_variations)
configs = experiment.create_variations(configs, seed_variations)

In [8]:
configs

[Config(script='/Users/kabu/Documents/workspace/rasa2/leaderboard/nlu/exp_0_stratify_intents.py', model=ModelConfig(name='interesting-config-name1', config_path='./config2.yml'), data=DataConfig(name='examples-rules-nlu', data_path='../../examples/rules/data/nlu.yml'), clear_rasa_cache=True, remove_data_for_intents_with_num_examples_below=2, train_exclusion_fraction=0.0, exclusion_seed=1, test_fraction=0.2, test_seed=42),
 Config(script='/Users/kabu/Documents/workspace/rasa2/leaderboard/nlu/exp_0_stratify_intents.py', model=ModelConfig(name='interesting-config-name1', config_path='./config2.yml'), data=DataConfig(name='examples-rules-nlu', data_path='../../examples/rules/data/nlu.yml'), clear_rasa_cache=True, remove_data_for_intents_with_num_examples_below=2, train_exclusion_fraction=0.0, exclusion_seed=2, test_fraction=0.2, test_seed=42),
 Config(script='/Users/kabu/Documents/workspace/rasa2/leaderboard/nlu/exp_0_stratify_intents.py', model=ModelConfig(name='interesting-config-name1',

**NOTE**: The following is done **automatically by the multi-run method below** before it actually executes the first config.

In [9]:
for config in configs:
    config.validate_no_missing()
    config.validate()

## Run!

In [10]:
!ls -lh ./

total 160
-rw-r--r--  1 kabu  staff    26K Apr  6 18:20 Example3_Collect-Results.ipynb
-rw-r--r--  1 kabu  staff    39K Apr  7 17:05 Run-Multiple-Intent-Experiments.ipynb
-rw-r--r--  1 kabu  staff     0B Mar 31 19:21 __init__.py
-rw-r--r--  1 kabu  staff   2.3K Apr  4 14:59 blub.csv
-rw-r--r--  1 kabu  staff   213B Mar 31 17:41 config1.yml
-rw-r--r--  1 kabu  staff   176B Apr  1 14:33 config2.yml
drwxr-xr-x  3 kabu  staff    96B Apr  5 13:56 test2
drwxr-xr-x  3 kabu  staff    96B Apr  6 10:53 test3


In [11]:
experiment.multirun(
    experiment_module = exp_0_stratify_intents,
    configs=configs, 
    out_dir='./test', 
    capture=True) # capture=True captures all stdout/stderr output

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [08:09<00:00, 61.22s/it]


## Inspect Output Folder

In [12]:
experiment_folder = "./test/exp_0_stratify_intents__examples-rules-nlu/"

In [13]:
!ls -lh {experiment_folder}

total 0
drwxr-xr-x  7 kabu  staff   224B Apr  7 17:10 model:another-very-interesting-config-because-reasons,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:1__2022-04-07_17-09-28
drwxr-xr-x  7 kabu  staff   224B Apr  7 17:11 model:another-very-interesting-config-because-reasons,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:2__2022-04-07_17-10-30
drwxr-xr-x  7 kabu  staff   224B Apr  7 17:12 model:another-very-interesting-config-because-reasons,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:3__2022-04-07_17-11-30
drwxr-xr-x  7 kabu  staff   224B Apr  7 17:13 model:another-very-interesting-config-because-reasons,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:4__2022-04-07_17-12-30
drwxr-xr-x  7 kabu  staff   224B Apr  7 17:06 model:interesting-config-name1,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:1__2022-04-07_17-05-22


In [14]:
import glob
example = list(glob.glob(experiment_folder + "*"))[0]

In [15]:
example

'./test/exp_0_stratify_intents__examples-rules-nlu/model:another-very-interesting-config-because-reasons,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:4__2022-04-07_17-12-30'

In [16]:
!ls -lha {example}

total 8
drwxr-xr-x   7 kabu  staff   224B Apr  7 17:13 .
drwxr-xr-x  10 kabu  staff   320B Apr  7 17:12 ..
drwxr-xr-x   5 kabu  staff   160B Apr  7 17:12 .hydra
drwxr-xr-x   4 kabu  staff   128B Apr  7 17:12 data
-rw-r--r--   1 kabu  staff   2.7K Apr  7 17:13 experiment.log
drwxr-xr-x   4 kabu  staff   128B Apr  7 17:12 model
drwxr-xr-x   8 kabu  staff   256B Apr  7 17:13 report


In [17]:
!ls -lha {example}/.rasa

ls: ./test/exp_0_stratify_intents__examples-rules-nlu/model:another-very-interesting-config-because-reasons,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:4__2022-04-07_17-12-30/.rasa: No such file or directory


In [18]:
!head -n 5 {example}/intent_experiment.log

head: ./test/exp_0_stratify_intents__examples-rules-nlu/model:another-very-interesting-config-because-reasons,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:4__2022-04-07_17-12-30/intent_experiment.log: No such file or directory


In [19]:
!tail -n 5 {example}/intent_experiment.log

tail: ./test/exp_0_stratify_intents__examples-rules-nlu/model:another-very-interesting-config-because-reasons,data:examples-rules-nlu,drop:2,exclude:0.0,test:0.2,test_seed:42,exclusion_seed:4__2022-04-07_17-12-30/intent_experiment.log: No such file or directory


In [20]:
!cat {example}/.hydra/config.yaml

script: /Users/kabu/Documents/workspace/rasa2/leaderboard/nlu/exp_0_stratify_intents.py
model:
  name: another-very-interesting-config-because-reasons
  config_path: ./config1.yml
data:
  name: examples-rules-nlu
  data_path: ../../examples/rules/data/nlu.yml
clear_rasa_cache: true
remove_data_for_intents_with_num_examples_below: 2
train_exclusion_fraction: 0.0
exclusion_seed: 4
test_fraction: 0.2
test_seed: 42


In [21]:
!head -n 5 {example}/.hydra/hydra.yaml
!echo '...'

hydra:
  run:
    dir: ./test/exp_0_stratify_intents__${data.name}/model:${model.name},data:${data.name},drop:${remove_data_for_intents_with_num_examples_below},exclude:${train_exclusion_fraction},test:${test_fraction},test_seed:${test_seed},exclusion_seed:${exclusion_seed}__${now:%Y-%m-%d_%H-%M-%S}
  sweep:
    dir: multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}
...


In [22]:
!ls {example}/report/

intent_confusion_matrix.png  intent_report.json
intent_errors.json           stats.csv
intent_histogram.png         training_metadata__times.csv


In [23]:
!cat {example}/report/training_metadata__times.csv

name,start_time,duration(min),duration(sec)
train_LexicalSyntacticFeaturizer1,2022-04-07 17:12:31,0.0003,0.019145965576171875
train_CountVectorsFeaturizer2,2022-04-07 17:12:31,0.0002,0.012251138687133789
train_DIETClassifier3,2022-04-07 17:12:32,0.3983,23.89854907989502


## Collect Results

In [24]:
from leaderboard.nlu import evaluation

In [25]:
import importlib
importlib.reload(evaluation)

<module 'leaderboard.nlu.evaluation' from '/Users/kabu/Documents/workspace/rasa2/leaderboard/nlu/evaluation.py'>

In [26]:
df_full = evaluation.results2df(experiment_folder)

In [27]:
df_full.drop(columns=["data"])

exp                                               \
                 data                  script            timestamp   
0  examples-rules-nlu  exp_0_stratify_intents  2022-04-07_17-12-30   
1  examples-rules-nlu  exp_0_stratify_intents  2022-04-07_17-07-29   
2  examples-rules-nlu  exp_0_stratify_intents  2022-04-07_17-10-30   
3  examples-rules-nlu  exp_0_stratify_intents  2022-04-07_17-11-30   
4  examples-rules-nlu  exp_0_stratify_intents  2022-04-07_17-09-28   
5  examples-rules-nlu  exp_0_stratify_intents  2022-04-07_17-08-29   
6  examples-rules-nlu  exp_0_stratify_intents  2022-04-07_17-06-25   
7  examples-rules-nlu  exp_0_stratify_intents  2022-04-07_17-05-22   

  intent_accuracy                              intent_affirm            \
                -                              confused_with  f1-score   
0        0.172414                 {'stop': 2, 'thankyou': 1}       0.0   
1        0.224138                                {'deny': 3}       0.0   
2        0.224138     {'inform': 2, 'request_restaurant': 1}       0.0   
3        0.275862               {'inform': 2, 'thankyou': 1}       0.0   
4        0.172414                            {'thankyou': 2}  0.222222   
5         0.12069  {'thankyou': 1, 'stop': 1, 'chitchat': 1}       0.0   
6         0.37931                 {'stop': 1, 'chitchat': 1}       0.4   
7        0.137931                {'greet': 1, 'chitchat': 1}       0.1   

                              intent_bot_challenge  ...  \
  precision    recall support        confused_with  ...   
0       0.0       0.0       3         {'greet': 1}  ...   
1       0.0       0.0       3        {'inform': 1}  ...   
2       0.0       0.0       3        {'inform': 1}  ...   
3       0.0       0.0       3        {'affirm': 1}  ...   
4  0.166667  0.333333       3      {'chitchat': 1}  ...   
5       0.0       0.0       3          {'deny': 1}  ...   
6       0.5  0.333333       3        {'inform': 1}  ...   
7  0.058824  0.333333       3         {'greet': 1}  ...   

  train_LexicalSyntacticFeaturizer1                                     \
                      duration(min) duration(sec)           start_time   
0                            0.0003      0.019146  2022-04-07 17:12:31   
1                               NaN           NaN                  NaN   
2                            0.0003       0.01866  2022-04-07 17:10:31   
3                            0.0003      0.018544  2022-04-07 17:11:31   
4                            0.0003      0.018332  2022-04-07 17:09:29   
5                               NaN           NaN                  NaN   
6                               NaN           NaN                  NaN   
7                               NaN           NaN                  NaN   

   train_all train_all_classifiers train_non_classifiers weighted avg  \
                                                             f1-score   
0  23.929946             23.898549              0.031397     0.175122   
1  23.858839             23.844899              0.013940     0.184182   
2  23.943153             23.912267              0.030886     0.236453   
3  23.765447             23.734897              0.030550     0.321073   
4  23.812527             23.782410              0.030117     0.165745   
5  23.466298             23.452626              0.013672     0.145544   
6  25.004739             24.991541              0.013198     0.380638   
7  24.632722             24.619096              0.013626     0.146758   

                               
  precision    recall support  
0  0.311864  0.172414    58.0  
1  0.285441  0.224138    58.0  
2  0.310345  0.224138    58.0  
3  0.469436  0.275862    58.0  
4   0.40546  0.172414    58.0  
5  0.253421   0.12069    58.0  
6  0.386761   0.37931    58.0  
7  0.295284  0.137931    58.0  

[8 rows x 88 columns]

In [28]:
df_full["intent_inform"]

,confused_with,f1-score,precision,recall,support
0,"{'request_restaurant': 10, 'greet': 3}",0.294118,0.625,0.192308,26
1,"{'request_restaurant': 8, 'deny': 7, 'greet': 4}",0.129032,0.4,0.076923,26
2,"{'request_restaurant': 9, 'affirm': 4}",0.461538,0.461538,0.461538,26
3,"{'deny': 5, 'bot_challenge': 4}",0.564103,0.846154,0.423077,26
4,"{'thankyou': 10, 'stop': 5, 'request_restauran...",0.193548,0.6,0.115385,26
5,"{'request_restaurant': 7, 'deny': 7, 'thankyou...",0.181818,0.428571,0.115385,26
6,"{'greet': 3, 'request_restaurant': 2}",0.705882,0.72,0.692308,26
7,"{'request_restaurant': 8, 'affirm': 6, 'deny': 4}",0.129032,0.4,0.076923,26


In [29]:
df_full['param']

,clear_rasa_cache,exclusion_seed,remove_data_for_intents_with_num_examples_below,script,test_fraction,test_seed,train_exclusion_fraction
0,True,4,2,/Users/kabu/Documents/workspace/rasa2/leaderbo...,0.2,42,0.0
1,True,3,2,/Users/kabu/Documents/workspace/rasa2/leaderbo...,0.2,42,0.0
2,True,2,2,/Users/kabu/Documents/workspace/rasa2/leaderbo...,0.2,42,0.0
3,True,3,2,/Users/kabu/Documents/workspace/rasa2/leaderbo...,0.2,42,0.0
4,True,1,2,/Users/kabu/Documents/workspace/rasa2/leaderbo...,0.2,42,0.0
5,True,4,2,/Users/kabu/Documents/workspace/rasa2/leaderbo...,0.2,42,0.0
6,True,2,2,/Users/kabu/Documents/workspace/rasa2/leaderbo...,0.2,42,0.0
7,True,1,2,/Users/kabu/Documents/workspace/rasa2/leaderbo...,0.2,42,0.0
